# Project Name : Chatbot using RNN

Due to some limitations of fed and forward artificial neural networks which use only present state not output of past state we have to use recurrent artificial neural networks in which output of past state or computation done by previous hidden layer can be used for current layer.For example , if we want to complete a sentence then  in that case previous words also matters suppose a sentence "please do the work" now what should be the next word , it may be "immediately". That is how chatbots work. There are many chatbots or assistances are in trend like apple's siri or cortana etc. all are using deep learning.
In this project , i am using concepts of recurrent artificial neural networks for building a chatbot . 

In [ ]:
import torch # using pytorch 
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [ ]:
device=torch.device("cpu") # i am using CPU because i don't have GPU . well , GPU can be used to increase processing speed .

# Part 1 : Data Processing

In [4]:
lines_filepath=os.path.join("cornell movie-dialogs corpus","movie_lines.txt")
corpus_name = "cornell movie-dialogs corpus"

In [5]:
conv_filepath=os.path.join("cornell movie-dialogs corpus","movie_conversations.txt")

In [6]:
# see some lines
with open(lines_filepath, 'rb') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line.strip())

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie."
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No'


In [7]:
#split each line in dictionary of files (lineID,characterID,movieID,character,dialog)
line_fields=["lineID","characterID","movieID","character","text"]
lines={}
with open(lines_filepath,'r',encoding='iso-8859-1') as f:
    for line in f:
        values=line.split(" +++$+++ ")
        lineObj={} # it is a temp dictionary for holding values of a line 
        for i, field in enumerate(line_fields):
            lineObj[field]=values[i]
        lines[lineObj['lineID']]=lineObj # key is lineID and value is full details 

In [8]:
lines

{'L1045': {'lineID': 'L1045',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': 'They do not!\n'},
 'L1044': {'lineID': 'L1044',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'They do to!\n'},
 'L985': {'lineID': 'L985',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': 'I hope so.\n'},
 'L984': {'lineID': 'L984',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'She okay?\n'},
 'L925': {'lineID': 'L925',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': "Let's go.\n"},
 'L924': {'lineID': 'L924',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'Wow\n'},
 'L872': {'lineID': 'L872',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': "Okay -- you're gonna need to learn how to lie.\n"},
 'L871': {'lineID': 'L871',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'No

In [9]:
# group fields of lines from loadlines into conversations based on "movie_conversations.txt"
conv_fields=["character1ID","character2ID","movieID","utteranceIDs"]
conversations=[]
with open(conv_filepath,'r',encoding='iso-8859-1') as f:
    for line in f:
        values=line.split(" +++$+++ ")
        convObj={} # it is a temp dictionary for holding values of a line 
        for i, field in enumerate(conv_fields):
            convObj[field]=values[i]
            
        lineIds=eval(convObj["utteranceIDs"])
        convObj["lines"]=[]
        for lineId in lineIds:
            convObj["lines"].append(lines[lineId])
        conversations.append(convObj)

In [10]:
conversations

[{'character1ID': 'u0',
  'character2ID': 'u2',
  'movieID': 'm0',
  'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n",
  'lines': [{'lineID': 'L194',
    'characterID': 'u0',
    'movieID': 'm0',
    'character': 'BIANCA',
    'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
   {'lineID': 'L195',
    'characterID': 'u2',
    'movieID': 'm0',
    'character': 'CAMERON',
    'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
   {'lineID': 'L196',
    'characterID': 'u0',
    'movieID': 'm0',
    'character': 'BIANCA',
    'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
   {'lineID': 'L197',
    'characterID': 'u2',
    'movieID': 'm0',
    'character': 'CAMERON',
    'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}]},
 {'character1ID': 'u0',
  'character2ID': 'u2',
  'movieID': 'm0',
  'uttera

In [11]:
# make pairs of sentences like QnA
qa_pairs=[]
for conversation in conversations:
    for i in range(len(conversation["lines"])-1):
        inputline=conversation["lines"][i]["text"].strip() # this is first statement or we can say it is a question
        targetline=conversation["lines"][i+1]["text"].strip() # this is second statement or we can say it is a reply
        if inputline and targetline:
            qa_pairs.append([inputline,targetline])

            

In [12]:
datafile=os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
delimiter='\t'
delimiter=str(codecs.decode(delimiter,"unicode_escape")) # unescape the delimiter
#write new csv file 
with open(datafile,'w',encoding='utf-8') as outputfile:
    writer=csv.writer(outputfile,delimiter=delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
print('done writing')


done writing


In [13]:
datafile=os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
with open(datafile,'rb') as file:  # rb is read-binary
    lines=file.readlines()
for line in lines[:8]:
    print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\n"
b'Why?\tUnsolved myster

In [14]:
PAD_token=0 # used for padding
SOS_token=1 # start of sentence
EOS_token=2 # end of sentence

class Vocabulary:
    def __init__(self,name):
        self.name=name # this is the name of voc
        self.word2index={} # this is dict for mapping word to unique index 
        self.word2count={}
        self.index2word={PAD_token: "PAD",SOS_token: "SOS", EOS_token: "EOS"}
        # index is key and word is value for example 0-PAD , 1-SOS , 2-EOS
        self.num_words=3
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word]=self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words]=word
            self.num_words+=1
        else:
            self.word2count[word]+=1
            
# remove words those are appearing below than thresold bcoz those will confuse our network

    def trim(self,min_count):
        keep_words=[]
        for k,v in self.word2count.items():
            if v>=min_count:
                keep_words.append(k)
        
        # reinitialize dict 
        
        self.word2index={} # this is dict for mapping word to unique index 
        self.word2count={}
        self.index2word={PAD_token: "PAD",SOS_token: "SOS", EOS_token: "EOS"}
        # index is key and word is value for example 0-PAD , 1-SOS , 2-EOS
        self.num_words=3
        
        for word in keep_words:
            self.addWord(word)

In [15]:
# convert a unicode string to ascii
def unicodetoAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c)!='Mn')

In [16]:
# normalize string 
# lowercase everything , remove space etc and non-letter charac..
# for our task we can regular expression module 
def normalizestring(s):
    s=unicodetoAscii(s.lower().strip())
    s=re.sub(r"([.!?])",r" \1",s) # . or ! or ? will be replaced by whitespace + char and r is to escape backslash
    s=re.sub(r"[^a-zA-Z.!?]+",r" ",s) # remove any char that is not from lower or upper case char
    s=re.sub(r"\s+",r" ",s).strip() # remove a sequence of more whitespaces 
    return s

In [17]:
normalizestring("aa123aa!s       dd?")

'aa aa !s dd ?'

In [18]:
datafile=os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
# read the file and split into lines 
lines=open(datafile,encoding='utf-8').read().strip().split('\n')
# split every line into pair and normalize 
pairs=[ [normalizestring(s) for s in pair.split('\t')] for pair in lines ]
voc=Vocabulary("cornell movie-dialogs corpus")


In [19]:
MAX_LENGTH=10 # max 10 words are allowed in a sentence 
def filterPair(p):
    return len(p[0].split()) < 10 and len(p[1].split())<10 # preserve last word for EOS
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]
save_dir = os.path.join("cornell movie-dialogs corpus", "save")


In [20]:
pairs=[pair for pair in pairs if len(pair)>1]
#print(len(pairs))  before filtering 
pairs=filterPairs(pairs)
#print(len(pairs)) after filtering

In [21]:
# loop through pairs and add those to voc
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print('counted words:',(voc.num_words))

counted words: 18008


In [22]:
# now trim the words which are not frequent 
MIN_COUNT=3

def trimRareWords(voc,pairs,MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs=[]
    for pair in pairs:
        input_sentence=pair[0]
        output_sentence=pair[1]
        keep_input=True
        keep_output=True
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input=False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output=False
                break
        if keep_input and keep_output:
            keep_pairs.append(pair)
        
    return keep_pairs
        
            

In [23]:
pairs=trimRareWords(voc,pairs,MIN_COUNT)

In [24]:
pairs
len(pairs)

53165

# Data Preaparation

In [25]:
def indexesFromSentence(voc,sentence):
    return [voc.word2index[word] for word in sentence.split(' ')]+[EOS_token]


In [26]:
indexesFromSentence(voc,pairs[1][0])

[7, 8, 9, 10, 4, 11, 12, 13, 2]

In [27]:
# testing
inp=[]
out=[]
for pair in pairs[:10]:
    inp.append(pair[0])
    out.append(pair[1])
print(inp)
indexes=[indexesFromSentence(voc,sentence) for sentence in inp]
indexes

['there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?', 'wow']


[[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [8, 31, 22, 6, 2],
 [33, 34, 4, 4, 4, 2],
 [35, 36, 37, 38, 7, 39, 40, 41, 4, 2],
 [42, 2],
 [47, 7, 48, 40, 45, 49, 6, 2],
 [50, 51, 52, 6, 2],
 [58, 2]]

In [28]:
def zeroPadding(l,fillvalue=0):
    return list(itertools.zip_longest(*l,fillvalue=fillvalue))

In [29]:
leng=[len(ind) for ind in indexes]
max(leng)


10

In [30]:
test_result=zeroPadding(indexes) 
print(len(test_result)) # max length is now no of rows 
test_result # test res where empty fields are filled with 0 

10


[(3, 7, 16, 8, 33, 35, 42, 47, 50, 58),
 (4, 8, 4, 31, 34, 36, 2, 7, 51, 2),
 (2, 9, 2, 22, 4, 37, 0, 48, 52, 0),
 (0, 10, 0, 6, 4, 38, 0, 40, 6, 0),
 (0, 4, 0, 2, 4, 7, 0, 45, 2, 0),
 (0, 11, 0, 0, 2, 39, 0, 49, 0, 0),
 (0, 12, 0, 0, 0, 40, 0, 6, 0, 0),
 (0, 13, 0, 0, 0, 41, 0, 2, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0)]

In [31]:
# converting the indexes to binary matrix 
def binaryMatrix(l,value=0):
    m=[]
    for i,seq in enumerate(l):
        m.append([])
        for token in seq:
            if token==PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)    
    return m
        

In [32]:
binary_result=binaryMatrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [33]:
#it is going to return padded input sequence tensor and 
# a tensor of lengths of each of sequences in the batch
def inputVar(l,voc):
    indexes_batch=[indexesFromSentence(voc,sentence) for sentence in l]
    lengths=torch.tensor([len(indexes) for indexes in indexes_batch])
    padList=zeroPadding(indexes_batch)
    padVar=torch.LongTensor(padList)
    return padVar,lengths

In [34]:
# returns padded target seq tensor , max lenght and mask 
def outputVar(l,voc):
    indexes_batch=[indexesFromSentence(voc,sentence) for sentence in l]
    max_target_len=max([len(indexes) for indexes in indexes_batch])
    padList=zeroPadding(indexes_batch)
    padVar=torch.LongTensor(padList)
    mask=binaryMatrix(padList)
    mask=torch.ByteTensor(mask)
    return padVar,mask,max_target_len

In [35]:
# return all item of each pair batch
def brain2TrainData(voc,pair_batch):
    pair_batch.sort(key=lambda x:len(x[0].split(" ")),reverse=True) #  sort in descending order
    input_batch,output_batch=[],[]
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp,lengths=inputVar(input_batch,voc)
    output,mask,max_target_len=outputVar(output_batch,voc)
    return inp,lengths,output,mask,max_target_len
    

In [36]:
# testing the function
small_batch_size=5
batches=brain2TrainData(voc,[random.choice(pairs) for _ in range(small_batch_size)])
input_variable,lengths,output_variable,mask,max_target_len=batches

print(input_variable)
print(lengths)
print(output_variable)
print(mask)
print(max_target_len)

tensor([[  25,   92,  401,  424,   50],
        [ 387,    7,  177,   76,   92],
        [4842,   38,   96,   40,    7],
        [  96,  266,    4,   83,  677],
        [  70,  367,    4,    4,    6],
        [5461,    6,    4,    2,    2],
        [   4,    2,    2,    0,    0],
        [   2,    0,    0,    0,    0]])
tensor([8, 7, 7, 6, 6])
tensor([[ 122,   25, 6880,  124, 6746],
        [  25,  102,    4,  159, 1462],
        [  96,  509, 6880,   61,    4],
        [ 945,  407, 7256,  115,    2],
        [   4,   76,    4,    4,    0],
        [   2,  479,    2,    2,    0],
        [   0, 2037,    0,    0,    0],
        [   0,    4,    0,    0,    0],
        [   0,    2,    0,    0,    0]])
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0]], dtype=torch.uint8)
9


In [37]:
# i am using seq 2 seq model here in which 2 RNNs have been used one is for encoding and another is for decoding 
# encoding == input_seq->fixed size context vector ( last hidden layer of RNN )
# decoding == predicting next word 

# i am using bi-directional GRU means putting the input seq in sequential order and in reverse order 
# advantage of bi-directional GRU is encoding both past and future context.

In [38]:
class EncoderRNN(nn.Module):
    def __init__(self,hidden_size,embedding,n_layers=1,dropout=0):
        super(EncoderRNN,self).__init__()
        self.n_layers=n_layers
        self.hidden_size=hidden_size
        self.embedding=embedding
        
        # input size is number of features 
        # initialize GRU where input size=hidden size
        self.gru=nn.GRU(hidden_size,hidden_size,n_layers,dropout=(0 if n_layers==1 else dropout), bidirectional=True)
    
    def forward(self,input_seq,input_lengths,hidden=None):
        #input_seq=batch of input sequence
        #input_lengths=list of sentence lengths corr. to each sentence in the batch
        embedded=self.embedding(input_seq)
        packed=torch.nn.utils.rnn.pack_padded_sequence(embedded,input_lengths) # pack padded seq batch for RNN module 
        outputs,hidden=self.gru(packed,hidden)
        outputs, _=torch.nn.utils.rnn.pad_packed_sequence(outputs)# unpack padding 
        outputs=outputs[:,:,:self.hidden_size]+outputs[:,:,self.hidden_size:]# bidirectional gru outputs 
        return outputs,hidden # return output and final hidden state
    
    
    
    

In [39]:
# Luong Attention Layer 
class Attn(torch.nn.Module):
    def __init__(self,method,hidden_size):
        super(Attn,self).__init__()
        self.method=method
        self.hidden_size=hidden_size
        
    def dot_score(self,hidden,encoder_output):
        return torch.sum(hidden*encoder_output,dim=2) # multiply current encoder state with encoder output and sum them
    
    def forward(self,hidden,encoder_outputs):
        # hidden of shape = (1,batch_size,max_length)
        # encoder outputs of shape = (max_length,batch_size,hidden_size)
        # dot product of (hidden of shape * encoder outputs of shape )
        # dot product = (max_length,batch_size,hidden_size)
        
        attn_energies=self.dot_score(hidden,encoder_outputs) # attn weights = attn energies # (max_length,batch_size)
        attn_energies=attn_energies.t() # (batch_size,max_length)
        return F.softmax(attn_energies,dim=1).unsqueeze(1) # softmax normalized probability scores 
         # (batch_size,1,max_length)
        

In [40]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self,attn_model,embedding,hidden_size,output_size,n_layers=1,dropout=0.1):
        super(LuongAttnDecoderRNN,self).__init__()
        self.attn_model=attn_model
        self.embedding=embedding
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.n_layers=n_layers
        self.dropout=dropout
        self.embedding_dropout=nn.Dropout(dropout)
        self.gru=nn.GRU(hidden_size,hidden_size,n_layers,dropout=(0 if n_layers==1 else dropout))
        self.concat=nn.Linear(hidden_size * 2 , hidden_size)
        self.out=nn.Linear(hidden_size,output_size)
        self.attn=Attn(attn_model,hidden_size)
        
    def forward(self,input_step,last_hidden,encoder_outputs):
        
        # input step = one time step (one word ) of input sequence batch
        # last hidden = final hidden state of GRU ; shape = (n_layers * num_directions , batch_size , hidden_size)
        # encoder_outputs = encoder model's output ; shape = (seq_len,batch,num_directions * hidden_size)
        # run one step(batch of words ) at a time 
        
        
        # get embedding of current input word 
        embedded=self.embedding(input_step)
        embedded=self.embedding_dropout(embedded)
        # forward through unidirectional GRU
        rnn_output,hidden=self.gru(embedded,last_hidden)
        # calculate attention weights from the current GRU output
        attn_weights=self.attn(rnn_output,encoder_outputs)
        # multiply attn weights to encoder outputs to get new "weighted sum" in context vector
        # (batch_size,1,max_length) bmm with (batch_size,max_length,hidden) = (batch_size,1,hidden)
        context=attn_weights.bmm(encoder_outputs.transpose(0,1))
        # concat weighted context vector and GRU output 
        
        rnn_output=rnn_output.squeeze(0)
        context=context.squeeze(1)
        concat_input=torch.cat((rnn_output,context),1)
        concat_output=torch.tanh(self.concat(concat_input))
        
        output=self.out(concat_output) # predict next word 
        output=F.softmax(output,dim=1)
        return output,hidden # return output and final hidden state 
        
        
        

# Training model


In [41]:
# # NLL-negative log likelihood LOSS

def maskNLLLoss(decoder_out,target,mask):
    nTotal=mask.sum() # non-zero elements 
    target=target.view(-1,1) # target size = (batch_size,1)
    gathered_tensor=torch.gather(decoder_out,1,target)
    crossEntropy=-torch.log(gathered_tensor)
    loss=crossEntropy.masked_select(mask)
    loss=loss.mean() # calculate the mean of the loss
    loss=loss.to(device)
    return loss, nTotal.item()

In [42]:
# teacher forcing - we are putting actual inputs or labels at each timestep 
# no teacher forcing - we are putting output of previous timestep to input , useful while testing , mimic real time situation

In [43]:
# visualizing what is happening in one iteration, only run this for visualization
small_batch_size=5
batches=brain2TrainData(voc,[random.choice(pairs) for _ in range(small_batch_size)])
input_variable,lengths,target_variable,mask,max_target_len=batches

print("input variable ",input_variable.shape)
print("lenghts ",lengths.shape)
print("target variable",target_variable.shape)
print("mask ",mask.shape)
print("max target len ",max_target_len)

# parameters 
hidden_size=500
encoder_n_layers=2
decoder_n_layers=2
dropout=0.1
attn_model='dot'
embedding=nn.Embedding(voc.num_words,hidden_size) # single word as feature 

# define encoder and decoder 

encoder=EncoderRNN(hidden_size,embedding,encoder_n_layers,dropout)
decoder=LuongAttnDecoderRNN(attn_model,embedding,hidden_size,voc.num_words,decoder_n_layers,dropout)
encoder=encoder.to(device)
decoder=decoder.to(device)

encoder.train()
decoder.train()

encoder_optimizer=optim.Adam(encoder.parameters(),lr=0.0001) # lr is learning rate 
decoder_optimizer=optim.Adam(decoder.parameters(),lr=0.0001)
encoder_optimizer.zero_grad()
decoder_optimizer.zero_grad()

input_variable=input_variable.to(device)
lengths=lengths.to(device)
target_variable=target_variable.to(device)
mask=mask.to(device)


loss=0
print_losses=[]
n_totals=0

encoder_outputs,encoder_hidden=encoder(input_variable,lengths)
print("encoder outputs ",encoder_outputs.shape)
print("encoder hidden ",encoder_hidden.shape)


decoder_input=torch.LongTensor([ [SOS_token for _ in range(small_batch_size)] ]  )
decoder_input=decoder_input.to(device)
print("decoder input shape ",decoder_input.shape)
print(decoder_input)

# set initial decoder hidden state to encoder's final hidden state

decoder_hidden=encoder_hidden[:decoder.n_layers]
print("initial decoder hidden state shape ",decoder_hidden.shape)
print('')
print("---------------------------------------------------------")
print("now let's look what's happening in every timestep of the GRU ")
print("----------------------------------------------------------")
print("")


# assume we are using teacher forcing

for t in range(max_target_len):
    decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden,encoder_outputs)
    print("decoder output shape ",decoder_output.shape)
    print("decoder hidden shape ",decoder_hidden.shape)
    # teacher forcing : next input is current target
    decoder_input=target_variable[t].view(1,-1)
    print("the target variable at the current timestep before reshaping ",target_variable[t])
    print("the target variable at the current timestep shape before reshaping ",target_variable[t].shape)
    print("the decoder input shape (reshape the target variable ) ",decoder_input.shape)
    # calculate and accumulate loss
    print("the mask at the current timestep ",mask[t])
    print("the mask at the current timestep shape",mask[t].shape)
    
    mask_loss,nTotal=maskNLLLoss(decoder_output,target_variable[t],mask[t])
    print("mask loss ",mask_loss)
    print("ntotal ",nTotal)
    
    loss+=mask_loss
    print_losses.append(mask_loss.item() * nTotal)
    print(print_losses)
    n_totals+=nTotal
    print(n_totals)
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    returned_loss=sum(print_losses)/n_totals
    print("Returned loss ",returned_loss)
    print('')
    print("--------done timestep------")

    
    
    
    
    
    

input variable  torch.Size([10, 5])
lenghts  torch.Size([5])
target variable torch.Size([9, 5])
mask  torch.Size([9, 5])
max target len  9
encoder outputs  torch.Size([10, 5, 500])
encoder hidden  torch.Size([4, 5, 500])
decoder input shape  torch.Size([1, 5])
tensor([[1, 1, 1, 1, 1]])
initial decoder hidden state shape  torch.Size([2, 5, 500])

---------------------------------------------------------
now let's look what's happening in every timestep of the GRU 
----------------------------------------------------------

decoder output shape  torch.Size([5, 7826])
decoder hidden shape  torch.Size([2, 5, 500])
the target variable at the current timestep before reshaping  tensor([424,   4,  25,  50, 371])
the target variable at the current timestep shape before reshaping  torch.Size([5])
the decoder input shape (reshape the target variable )  torch.Size([1, 5])
the mask at the current timestep  tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
the mask at the current timestep shape torch.Size(

In [44]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals



# Training iterations


In [45]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [brain2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

# Define Evaluation


In [46]:
# using Greedy Decoding 
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores


In [47]:
# evaluate 

def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizestring(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

# Run Model



In [48]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


# Run Training


In [49]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0005
decoder_learning_ratio = 5.0
n_iteration = 25000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)


Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.9475
Iteration: 2; Percent complete: 0.0%; Average loss: 8.0594
Iteration: 3; Percent complete: 0.0%; Average loss: 6.3776
Iteration: 4; Percent complete: 0.0%; Average loss: 5.3263
Iteration: 5; Percent complete: 0.0%; Average loss: 4.7962
Iteration: 6; Percent complete: 0.0%; Average loss: 4.8089
Iteration: 7; Percent complete: 0.0%; Average loss: 4.9265
Iteration: 8; Percent complete: 0.0%; Average loss: 4.7391
Iteration: 9; Percent complete: 0.0%; Average loss: 4.3612
Iteration: 10; Percent complete: 0.0%; Average loss: 4.5601
Iteration: 11; Percent complete: 0.0%; Average loss: 4.4068
Iteration: 12; Percent complete: 0.0%; Average loss: 4.6537
Iteration: 13; Percent complete: 0.1%; Average loss: 4.3096
Iteration: 14; Percent complete: 0.1%; Average loss: 4.1777
Iteration: 15; Percent complete: 0.1%; Average loss: 4.4784
Iteration: 16; Percent complete: 0.1%

Iteration: 136; Percent complete: 0.5%; Average loss: 3.7063
Iteration: 137; Percent complete: 0.5%; Average loss: 3.6837
Iteration: 138; Percent complete: 0.6%; Average loss: 3.5605
Iteration: 139; Percent complete: 0.6%; Average loss: 3.4887
Iteration: 140; Percent complete: 0.6%; Average loss: 3.2031
Iteration: 141; Percent complete: 0.6%; Average loss: 3.6360
Iteration: 142; Percent complete: 0.6%; Average loss: 3.4034
Iteration: 143; Percent complete: 0.6%; Average loss: 3.5861
Iteration: 144; Percent complete: 0.6%; Average loss: 3.6953
Iteration: 145; Percent complete: 0.6%; Average loss: 3.1870
Iteration: 146; Percent complete: 0.6%; Average loss: 3.5541
Iteration: 147; Percent complete: 0.6%; Average loss: 3.4825
Iteration: 148; Percent complete: 0.6%; Average loss: 3.8110
Iteration: 149; Percent complete: 0.6%; Average loss: 3.4633
Iteration: 150; Percent complete: 0.6%; Average loss: 3.5766
Iteration: 151; Percent complete: 0.6%; Average loss: 3.4271
Iteration: 152; Percent 

Iteration: 271; Percent complete: 1.1%; Average loss: 3.0462
Iteration: 272; Percent complete: 1.1%; Average loss: 3.3695
Iteration: 273; Percent complete: 1.1%; Average loss: 3.3336
Iteration: 274; Percent complete: 1.1%; Average loss: 3.2912
Iteration: 275; Percent complete: 1.1%; Average loss: 3.5213
Iteration: 276; Percent complete: 1.1%; Average loss: 3.2460
Iteration: 277; Percent complete: 1.1%; Average loss: 3.4076
Iteration: 278; Percent complete: 1.1%; Average loss: 3.5415
Iteration: 279; Percent complete: 1.1%; Average loss: 2.9570
Iteration: 280; Percent complete: 1.1%; Average loss: 3.3716
Iteration: 281; Percent complete: 1.1%; Average loss: 3.0498
Iteration: 282; Percent complete: 1.1%; Average loss: 3.2521
Iteration: 283; Percent complete: 1.1%; Average loss: 3.2479
Iteration: 284; Percent complete: 1.1%; Average loss: 3.2844
Iteration: 285; Percent complete: 1.1%; Average loss: 3.4908
Iteration: 286; Percent complete: 1.1%; Average loss: 3.0516
Iteration: 287; Percent 

Iteration: 406; Percent complete: 1.6%; Average loss: 3.4131
Iteration: 407; Percent complete: 1.6%; Average loss: 3.2528
Iteration: 408; Percent complete: 1.6%; Average loss: 3.0962
Iteration: 409; Percent complete: 1.6%; Average loss: 3.1176
Iteration: 410; Percent complete: 1.6%; Average loss: 3.3625
Iteration: 411; Percent complete: 1.6%; Average loss: 3.0901
Iteration: 412; Percent complete: 1.6%; Average loss: 3.2543
Iteration: 413; Percent complete: 1.7%; Average loss: 3.2240
Iteration: 414; Percent complete: 1.7%; Average loss: 3.1515
Iteration: 415; Percent complete: 1.7%; Average loss: 3.3172
Iteration: 416; Percent complete: 1.7%; Average loss: 3.3511
Iteration: 417; Percent complete: 1.7%; Average loss: 3.4260
Iteration: 418; Percent complete: 1.7%; Average loss: 3.1131
Iteration: 419; Percent complete: 1.7%; Average loss: 3.1549
Iteration: 420; Percent complete: 1.7%; Average loss: 3.0259
Iteration: 421; Percent complete: 1.7%; Average loss: 3.1554
Iteration: 422; Percent 

Iteration: 541; Percent complete: 2.2%; Average loss: 2.7069
Iteration: 542; Percent complete: 2.2%; Average loss: 2.9150
Iteration: 543; Percent complete: 2.2%; Average loss: 3.0062
Iteration: 544; Percent complete: 2.2%; Average loss: 2.9199
Iteration: 545; Percent complete: 2.2%; Average loss: 2.9569
Iteration: 546; Percent complete: 2.2%; Average loss: 2.7919
Iteration: 547; Percent complete: 2.2%; Average loss: 3.2665
Iteration: 548; Percent complete: 2.2%; Average loss: 2.8697
Iteration: 549; Percent complete: 2.2%; Average loss: 3.2798
Iteration: 550; Percent complete: 2.2%; Average loss: 3.1536
Iteration: 551; Percent complete: 2.2%; Average loss: 3.2325
Iteration: 552; Percent complete: 2.2%; Average loss: 2.9953
Iteration: 553; Percent complete: 2.2%; Average loss: 3.0402
Iteration: 554; Percent complete: 2.2%; Average loss: 3.1176
Iteration: 555; Percent complete: 2.2%; Average loss: 3.0288
Iteration: 556; Percent complete: 2.2%; Average loss: 3.0500
Iteration: 557; Percent 

...

Iteration: 24456; Percent complete: 97.8%; Average loss: 1.2850
Iteration: 24457; Percent complete: 97.8%; Average loss: 1.0949
Iteration: 24458; Percent complete: 97.8%; Average loss: 1.0181
Iteration: 24459; Percent complete: 97.8%; Average loss: 0.8923
Iteration: 24460; Percent complete: 97.8%; Average loss: 0.8627
Iteration: 24461; Percent complete: 97.8%; Average loss: 0.9521
Iteration: 24462; Percent complete: 97.8%; Average loss: 1.0579
Iteration: 24463; Percent complete: 97.9%; Average loss: 0.9379
Iteration: 24464; Percent complete: 97.9%; Average loss: 1.0144
Iteration: 24465; Percent complete: 97.9%; Average loss: 1.2128
Iteration: 24466; Percent complete: 97.9%; Average loss: 1.0470
Iteration: 24467; Percent complete: 97.9%; Average loss: 1.2951
Iteration: 24468; Percent complete: 97.9%; Average loss: 1.1078
Iteration: 24469; Percent complete: 97.9%; Average loss: 0.9957
Iteration: 24470; Percent complete: 97.9%; Average loss: 0.9918
Iteration: 24471; Percent complete: 97.9

Iteration: 24585; Percent complete: 98.3%; Average loss: 0.9170
Iteration: 24586; Percent complete: 98.3%; Average loss: 1.0321
Iteration: 24587; Percent complete: 98.3%; Average loss: 1.1945
Iteration: 24588; Percent complete: 98.4%; Average loss: 1.2843
Iteration: 24589; Percent complete: 98.4%; Average loss: 0.9320
Iteration: 24590; Percent complete: 98.4%; Average loss: 1.0888
Iteration: 24591; Percent complete: 98.4%; Average loss: 1.0387
Iteration: 24592; Percent complete: 98.4%; Average loss: 1.0407
Iteration: 24593; Percent complete: 98.4%; Average loss: 0.8240
Iteration: 24594; Percent complete: 98.4%; Average loss: 0.8510
Iteration: 24595; Percent complete: 98.4%; Average loss: 1.0180
Iteration: 24596; Percent complete: 98.4%; Average loss: 1.0815
Iteration: 24597; Percent complete: 98.4%; Average loss: 0.9826
Iteration: 24598; Percent complete: 98.4%; Average loss: 1.1539
Iteration: 24599; Percent complete: 98.4%; Average loss: 1.0644
Iteration: 24600; Percent complete: 98.4

Iteration: 24714; Percent complete: 98.9%; Average loss: 0.9396
Iteration: 24715; Percent complete: 98.9%; Average loss: 1.2781
Iteration: 24716; Percent complete: 98.9%; Average loss: 0.9765
Iteration: 24717; Percent complete: 98.9%; Average loss: 1.0027
Iteration: 24718; Percent complete: 98.9%; Average loss: 1.0365
Iteration: 24719; Percent complete: 98.9%; Average loss: 0.9994
Iteration: 24720; Percent complete: 98.9%; Average loss: 1.0432
Iteration: 24721; Percent complete: 98.9%; Average loss: 1.4139
Iteration: 24722; Percent complete: 98.9%; Average loss: 1.1619
Iteration: 24723; Percent complete: 98.9%; Average loss: 1.1941
Iteration: 24724; Percent complete: 98.9%; Average loss: 0.9369
Iteration: 24725; Percent complete: 98.9%; Average loss: 1.2351
Iteration: 24726; Percent complete: 98.9%; Average loss: 1.1028
Iteration: 24727; Percent complete: 98.9%; Average loss: 1.5220
Iteration: 24728; Percent complete: 98.9%; Average loss: 1.1353
Iteration: 24729; Percent complete: 98.9

Iteration: 24843; Percent complete: 99.4%; Average loss: 1.1215
Iteration: 24844; Percent complete: 99.4%; Average loss: 0.9912
Iteration: 24845; Percent complete: 99.4%; Average loss: 0.7862
Iteration: 24846; Percent complete: 99.4%; Average loss: 0.9708
Iteration: 24847; Percent complete: 99.4%; Average loss: 1.1647
Iteration: 24848; Percent complete: 99.4%; Average loss: 1.1856
Iteration: 24849; Percent complete: 99.4%; Average loss: 1.1537
Iteration: 24850; Percent complete: 99.4%; Average loss: 1.0466
Iteration: 24851; Percent complete: 99.4%; Average loss: 1.0272
Iteration: 24852; Percent complete: 99.4%; Average loss: 1.1012
Iteration: 24853; Percent complete: 99.4%; Average loss: 0.9718
Iteration: 24854; Percent complete: 99.4%; Average loss: 1.1498
Iteration: 24855; Percent complete: 99.4%; Average loss: 0.9185
Iteration: 24856; Percent complete: 99.4%; Average loss: 0.8533
Iteration: 24857; Percent complete: 99.4%; Average loss: 1.1536
Iteration: 24858; Percent complete: 99.4

Iteration: 24972; Percent complete: 99.9%; Average loss: 1.1705
Iteration: 24973; Percent complete: 99.9%; Average loss: 1.0300
Iteration: 24974; Percent complete: 99.9%; Average loss: 1.0351
Iteration: 24975; Percent complete: 99.9%; Average loss: 0.8419
Iteration: 24976; Percent complete: 99.9%; Average loss: 0.9854
Iteration: 24977; Percent complete: 99.9%; Average loss: 1.3851
Iteration: 24978; Percent complete: 99.9%; Average loss: 1.3281
Iteration: 24979; Percent complete: 99.9%; Average loss: 0.9194
Iteration: 24980; Percent complete: 99.9%; Average loss: 1.2411
Iteration: 24981; Percent complete: 99.9%; Average loss: 1.1371
Iteration: 24982; Percent complete: 99.9%; Average loss: 0.8842
Iteration: 24983; Percent complete: 99.9%; Average loss: 0.9280
Iteration: 24984; Percent complete: 99.9%; Average loss: 1.1910
Iteration: 24985; Percent complete: 99.9%; Average loss: 0.9407
Iteration: 24986; Percent complete: 99.9%; Average loss: 1.1334
Iteration: 24987; Percent complete: 99.9

In [ ]:
# Set dropout layers to eval mode

# encoder.eval()
# decoder.eval()

# searcher = GreedySearchDecoder(encoder, decoder) # Initialize search module

# Begin chatting (uncomment and run the following line to begin)
# evaluateInput(encoder, decoder, searcher, voc)